<a href="https://colab.research.google.com/github/Abdulaziz-06/PYTHON-_LEARNING/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
uploaded = files.upload()

import pandas as pd

data = pd.read_csv('ner_dataset.csv', encoding='unicode_escape')
data.head()


Saving ner_dataset.csv to ner_dataset (1).csv


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [7]:
def get_dict_map(data, token_or_tag):
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].dropna()))
    else:
        vocab = list(set(data['Tag'].dropna()))

    tok2idx = {tok: idx for idx, tok in enumerate(vocab)}
    idx2tok = {idx: tok for tok, idx in tok2idx.items()}

    return tok2idx, idx2tok

token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')


In [8]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data.fillna(method='ffill', inplace=True)

# Use double square brackets for multiple columns (FutureWarning fix)
data_group = data.groupby('Sentence #', as_index=False)[
    ['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx']
].agg(lambda x: list(x))


<ipython-input-8-3468b8f63ca5>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [9]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def get_pad_train_test_val(data_group, data, tag2idx):
    n_token = len(set(data['Word']))
    n_tag = len(set(data['Tag']))

    tokens = data_group['Word_idx'].tolist()
    maxlen = max(len(seq) for seq in tokens)
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, padding='post', value=n_token-1)

    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, padding='post', value=tag2idx["O"])
    pad_tags = [to_categorical(i, num_classes=n_tag) for i in pad_tags]

    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_, tags_, test_size=0.25, random_state=2020)

    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags, maxlen, n_token, n_tag

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags, maxlen, input_dim, n_tags = get_pad_train_test_val(data_group, data, tag2idx)


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Dropout, Bidirectional

output_dim = 64

def get_bilstm_lstm_model(input_dim, output_dim, input_length, n_tags):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim+1, output_dim=output_dim, input_length=input_length))
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

model = get_bilstm_lstm_model(input_dim, output_dim, maxlen, n_tags)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
import numpy as np

def train_model(X, y, model, epochs=5):
    history = model.fit(
        X, np.array(y),
        batch_size=1000,
        validation_split=0.2,
        epochs=epochs,
        verbose=1
    )
    return history

history = train_model(train_tokens, train_tags, model, epochs=25)


Epoch 1/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 178s 6s/step - accuracy: 0.8025 - loss: 2.0886 - val_accuracy: 0.9681 - val_loss: 0.4525
Epoch 2/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 185s 5s/step - accuracy: 0.9679 - loss: 0.4408 - val_accuracy: 0.9681 - val_loss: 0.3528
Epoch 3/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 143s 5s/step - accuracy: 0.9674 - loss: 0.3761 - val_accuracy: 0.9681 - val_loss: 0.3115
Epoch 4/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 211s 6s/step - accuracy: 0.9673 - loss: 0.3426 - val_accuracy: 0.9681 - val_loss: 0.2697
Epoch 5/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 188s 5s/step - accuracy: 0.9677 - loss: 0.2923 - val_accuracy: 0.9681 - val_loss: 0.2100
Epoch 6/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 143s 5s/step - accuracy: 0.9669 - loss: 0.2554 - val_accuracy: 0.9666 - val_loss: 0.2299
Epoch 7/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 146s 6s/step - accuracy: 0.9664 - loss: 0.2503 - val_accuracy: 0.9680 - val_loss: 0.2169
Epoch 8/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 203s 5s/step - accuracy: 0.9673 - loss: 0.2285 - val_accuracy: 0.9680 - v

In [13]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_sm')
text = nlp("Hi, My name is Abdul Aziz. I am excited to work on topics like cv. Steve Jobs is my inspiration.")
displacy.render(text, style='ent', jupyter=True)
